In [1]:
import os
import random
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

sys.path.append("..")
import warnings

import wandb
from otc.metrics.metrics import effective_spread
from scipy.stats import wilcoxon
from tqdm.auto import tqdm


/pfs/data5/home/kit/stud/uloak/thesis/thesis/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# set here globally
EXCHANGE = "ise"
MODELS = ["gbm", "classical"]  # ["gbm", "classical"]
SUBSET = "test"  # "all"
STRATEGY = "supervised"  # "transfer"


In [3]:
KEY = f"{EXCHANGE}_{STRATEGY}_{SUBSET}"
DATASET = f"fbv/thesis/{EXCHANGE}_{STRATEGY}_raw:latest"

os.environ["GCLOUD_PROJECT"] = "flowing-mantis-239216"

run = wandb.init(project="thesis", entity="fbv")

# load unscaled data
artifact = run.use_artifact(DATASET)  # type: ignore
data_dir = artifact.download()

# load results
result_dirs = []
for model in MODELS:
    results = f"fbv/thesis/{EXCHANGE}_{model}_{STRATEGY}_{SUBSET}:latest"
    artifact = run.use_artifact(results)  # type: ignore
    result_dir = artifact.download()
    result_dirs.append(result_dir)


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: karelze (fbv). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact ise_supervised_raw:latest, 2589.45MB. 3 files... 
wandb:   3 of 3 files downloaded.  
Done. 0:0:1.2
wandb: Downloading large artifact ise_gbm_supervised_test:latest, 129.80MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0
wandb: Downloading large artifact ise_classical_supervised_test:latest, 291.21MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.0


In [4]:
# p. 35-38
COLUMNS = [
    "buy_sell",
    "EXPIRATION",
    "QUOTE_DATETIME",
    "TRADE_SIZE",
    "TRADE_PRICE",
    "ask_ex",
    "bid_ex",
    "myn",
    "OPTION_TYPE",
    "issue_type",
]


if SUBSET == "all":
    train = pd.read_parquet(
        Path(data_dir, "train_set"), engine="fastparquet", columns=COLUMNS
    )
    val = pd.read_parquet(
        Path(data_dir, "val_set"), engine="fastparquet", columns=COLUMNS
    )
    test = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=COLUMNS
    )
    eval_data = pd.concat([train, val, test])
    del train, val, test

elif SUBSET == "test":
    eval_data = pd.read_parquet(
        Path(data_dir, "test_set"), engine="fastparquet", columns=COLUMNS
    )


results = []
for i, model in tqdm(enumerate(MODELS)):
    result = pd.read_parquet(Path(result_dirs[i], "results"), engine="fastparquet")
    result.columns = pd.MultiIndex.from_product([[model], result.columns])
    results.append(result)

results_data = pd.concat(results, axis=1, names=MODELS)

assert len(eval_data) == len(results_data)

X_print = eval_data

del results


2it [00:03,  1.63s/it]


In [5]:
results_data.shape


(9861576, 29)

In [6]:
# FIXME: select a subset of results for testing.
results_data = results_data[
    [
        ("gbm", "gbm(classical)"),
        ("gbm", "gbm(classical-retraining)"),
        ("gbm", "gbm(classical-size-retraining)"),
        ("gbm", "gbm(classical-size)"),
        ("gbm", "gbm(ml)"),
        ("gbm", "gbm(ml-retraining)"),
        ("gbm", "gbm(semi-classical)"),
        # ("gbm",'gbm(semi-classical-retraining)'),
        ("classical", "tick(ex)"),
        ("classical", "quote(ex)"),
        ("classical", "lr(ex)"),
        ("classical", "emo(ex)"),
        ("classical", "clnv(ex)"),
        (
            "classical",
            "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)",
        ),
    ]
]


In [7]:
results_data


gbm                            \
         gbm(classical) gbm(classical-retraining)   
index                                               
39342191             -1                        -1   
39342190             -1                        -1   
39342189              1                         1   
39342188             -1                        -1   
39342187             -1                        -1   
...                 ...                       ...   
49203742             -1                        -1   
49203743              1                         1   
49203744              1                         1   
49203745              1                         1   
49203746              1                         1   

                                                                     \
         gbm(classical-size-retraining) gbm(classical-size) gbm(ml)   
index                                                                 
39342191                             -1                  -1      -1   
39342190                             -1                  -1      -1   
39342189                              1                   1       1   
39342188                             -1                  -1      -1   
39342187                             -1                  -1      -1   
...                                 ...                 ...     ...   
49203742                              1                   1      -1   
49203743                              1                   1       1   
49203744                              1                   1       1   
49203745                              1                   1       1   
49203746                              1                   1       1   

                                                classical                   \
         gbm(ml-retraining) gbm(semi-classical)  tick(ex) quote(ex) lr(ex)   
index                                                                        
39342191                 -1                  -1       1.0      -1.0   -1.0   
39342190                 -1                  -1      -1.0      -1.0   -1.0   
39342189                  1                   1       1.0       1.0    1.0   
39342188                 -1                  -1      -1.0      -1.0   -1.0   
39342187                 -1                  -1       1.0      -1.0   -1.0   
...                     ...                 ...       ...       ...    ...   
49203742                 -1                  -1      -1.0       1.0    1.0   
49203743                  1                   1      -1.0       1.0    1.0   
49203744                  1                   1      -1.0      -1.0   -1.0   
49203745                  1                   1      -1.0       1.0    1.0   
49203746                  1                   1       1.0       1.0    1.0   

                           \
         emo(ex) clnv(ex)   
index                       
39342191    -1.0     -1.0   
39342190    -1.0     -1.0   
39342189     1.0      1.0   
39342188    -1.0     -1.0   
39342187    -1.0     -1.0   
...          ...      ...   
49203742     1.0      1.0   
49203743    -1.0      1.0   
49203744    -1.0     -1.0   
49203745     1.0      1.0   
49203746     1.0      1.0   

                                                                                        
         trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)  
index                                                                                   
39342191                                               -1.0                             
39342190                                               -1.0                             
39342189                                                1.0                             
39342188                                               -1.0                             
39342187                                               -1.0                             
...                                                     ...           

In [8]:
LUT = {
    "Trade_Size(ex)->Quote(Best)->Depth(Best)->Quote(Ex)->Depth(Ex)->Rev_Tick(All)": "\gls{GBM}",
    "(Ex)": " (Ex)",
    "(Best)": " (Best)",
    "(Classical)": " (Classical)",
    "(Classical-Size)": " (Classical, Size)",
    "Rev_": "Rev. ",
    "Trade_Size": "Trade Size",
    "Depth": "Depth",
    "->": " $\\to$ ",
    "Lr": "\gls{LR}",
    "Emo": "\gls{EMO}",
    "Clnv": "\gls{CLNV}",
    "OPTION_TYPE": "Option Type",
    "_": "$\_",
    "Gbm": "\gls{GBM}",
}

LUT_INDEX = {
    "OPTION_TYPE": "Option Type",
    "issue_type": "Security Type",
    "TRADE_SIZE_binned": "Trade Size",
    "year_binned": "Year",
    "ttm_binned": "Time to Maturity",
    "myn_binned": "Moneyness",
    "prox_q_binned": "Location to Quote",
    "all": "All trades",
}


def cell_str(x):
    x = x.title()
    for orig, sub in LUT.items():
        x = x.replace(orig, sub)
    # title-case everything
    return x


def highlight_max(s, props=""):
    return np.where(s == np.nanmax(s.values), props, "")


In [9]:
def set_tex_style(styler, caption, label, bold_axis=1):
    res = styler.set_caption(caption)

    res = (
        res.apply(highlight_max, props="font-weight:bold;", axis=bold_axis)
        .format(precision=4, decimal=".", thousands=",", escape=False, hyperlinks=None)
        .format_index(cell_str, axis=0)
        .format_index(cell_str, axis=1)
        .to_latex(
            f"{label}.tex",
            siunitx=True,
            position_float="centering",
            hrules=True,
            clines="skip-last;data",
            label="tab:" + label,
            caption=caption,
            convert_css=True,
        )
    )
    return res


In [10]:
classifiers = results_data.columns.tolist()
criterions = list(LUT_INDEX)


## Unclassified by method

In [11]:
results_data.head()


gbm                            \
         gbm(classical) gbm(classical-retraining)   
index                                               
39342191             -1                        -1   
39342190             -1                        -1   
39342189              1                         1   
39342188             -1                        -1   
39342187             -1                        -1   

                                                                     \
         gbm(classical-size-retraining) gbm(classical-size) gbm(ml)   
index                                                                 
39342191                             -1                  -1      -1   
39342190                             -1                  -1      -1   
39342189                              1                   1       1   
39342188                             -1                  -1      -1   
39342187                             -1                  -1      -1   

                                                classical                   \
         gbm(ml-retraining) gbm(semi-classical)  tick(ex) quote(ex) lr(ex)   
index                                                                        
39342191                 -1                  -1       1.0      -1.0   -1.0   
39342190                 -1                  -1      -1.0      -1.0   -1.0   
39342189                  1                   1       1.0       1.0    1.0   
39342188                 -1                  -1      -1.0      -1.0   -1.0   
39342187                 -1                  -1       1.0      -1.0   -1.0   

                           \
         emo(ex) clnv(ex)   
index                       
39342191    -1.0     -1.0   
39342190    -1.0     -1.0   
39342189     1.0      1.0   
39342188    -1.0     -1.0   
39342187    -1.0     -1.0   

                                                                                        
         trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)  
index                                                                                   
39342191                                               -1.0                             
39342190                                               -1.0                             
39342189                                                1.0                             
39342188                                               -1.0                             
39342187                                               -1.0

In [12]:
unclassified = (
    (results_data[results_data == 0.0].count(axis=0) / len(results_data.index))
    .sort_values(ascending=False)
    .to_frame(name="unclassified")
)


In [13]:
unclassified.style.pipe(
    set_tex_style,
    caption=(f"{KEY}-unclassified-long", "{key}-unclassified-short"),
    label=f"{KEY.lower()}-unclassfied",
    bold_axis=0,
)
unclassified


unclassified
classical tick(ex)                                                0.124762
          quote(ex)                                               0.060195
          emo(ex)                                                 0.023059
          clnv(ex)                                                0.019444
          lr(ex)                                                  0.001618
          trade_size(ex)->quote(best)->depth(best)->quote...      0.000043
gbm       gbm(classical)                                          0.000000
          gbm(classical-retraining)                               0.000000
          gbm(classical-size-retraining)                          0.000000
          gbm(classical-size)                                     0.000000
          gbm(ml)                                                 0.000000
          gbm(ml-retraining)                                      0.000000
          gbm(semi-classical)                                     0.000000

## Fill in unclassified

Unclassified are `0`.

In [14]:
rng = np.random.default_rng(42)

# replace 0 -> nan -> [-1,1]
results_data.replace(0, np.nan)
filler = pd.DataFrame(
    rng.choice(a=[-1, 1], size=results_data.shape),
    index=results_data.index,
    columns=results_data.columns,
)
results_data.fillna(filler, inplace=True)


### Robustness Checks

In [15]:
# prepare columns for printing
X_print["ttm"] = (
    X_print["EXPIRATION"].dt.to_period("M")
    - X_print["QUOTE_DATETIME"].dt.to_period("M")
).apply(lambda x: x.n)

X_print["year"] = X_print["QUOTE_DATETIME"].dt.year

bins_tradesize = [-1, 1, 3, 5, 11, np.inf]
trade_size_labels = ["(0,1]", "(1,3]", "(3,5]", "(5,11]", ">11"]
X_print["TRADE_SIZE_binned"] = pd.cut(
    X_print["TRADE_SIZE"], bins_tradesize, labels=trade_size_labels
)

# p. 38
bins_years = [2004, 2007, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017]
year_labels = [
    "2005-2007",
    "2008-2010",
    "2011",
    "2012",
    "2013",
    "2014",
    "2015",
    "2016",
    "2017",
]
X_print["year_binned"] = pd.cut(X_print["year"], bins_years, labels=year_labels)

# p. 37
bins_ttm = [-1, 1, 2, 3, 6, 12, np.inf]
ttm_labels = [
    "<= 1",
    "(1-2]",
    "(2-3]",
    "(3-6]",
    "(6-12]",
    "> 12",
]
X_print["ttm_binned"] = pd.cut(X_print["ttm"], bins_ttm, labels=ttm_labels)

# Security type
# see 3.0a-mb-explanatory-data-analysis.ipynb
X_print["issue_type"] = X_print["issue_type"].map(
    {
        "0": "Stock option",
        "A": "Index option",
        "7": "Others",
        "F": "Others",
        "%": "Others",
        " ": "Others",
    }
)

# Moneyness p. 38
bins_myn = [-1, 0.7, 0.9, 1.1, 1.3, np.inf]
myn_labels = [
    "<= 0.7 c",
    "(0.7-0.9]",
    "(0.9-1.1]",
    "(1.1-1.3]",
    "> 1.3",
]
X_print["myn_binned"] = pd.cut(X_print["myn"], bins_myn, labels=myn_labels)

# mid p. 31 + extra category for unknowns
ask = X_print["ask_ex"]
bid = X_print["bid_ex"]
trade_price = X_print["TRADE_PRICE"]

mid = np.where(ask >= bid, (ask + bid) * 0.5, np.nan)

prox_quotes = np.where(
    trade_price == mid,
    0,  # at mid
    np.where(
        (ask < trade_price) & (trade_price < bid),
        1,  # inside
        np.where(
            (trade_price == bid) | (ask == trade_price),
            2,  # at quotes
            np.where((trade_price < bid) | (ask > trade_price), 3, 4),
        ),
    ),
)  # outside + unclassifiable

bins_prox = [-np.inf, 0, 1, 2, 3, 4]
prox_labels = [
    "at mid",
    "inside",
    "at quotes",
    "outside",
    "unknown",
]

X_print["prox_q_binned"] = pd.cut(prox_quotes, bins_prox, labels=prox_labels)
X_print["mid"] = mid

# clean up empty buckets, as it causes empty grouping in result set generation
X_print["year_binned"] = X_print["year_binned"].cat.remove_unused_categories()
X_print["myn_binned"] = X_print["myn_binned"].cat.remove_unused_categories()
X_print["ttm_binned"] = X_print["ttm_binned"].cat.remove_unused_categories()
X_print["prox_q_binned"] = X_print["prox_q_binned"].cat.remove_unused_categories()

X_print["all"] = "all"

X_print.drop(
    columns=[
        "EXPIRATION",
        "QUOTE_DATETIME",
        "TRADE_SIZE",
        "ttm",
        "myn",
        "ask_ex",
        "bid_ex",
        "year",
    ],
    inplace=True,
)


In [16]:
X_print = pd.concat([X_print, results_data], axis=1)


In [17]:
X_print.head().T


index,39342191,39342190,39342189,39342188,39342187
buy_sell,-1,-1,1,-1,-1
TRADE_PRICE,3.5,6.38,0.13,0.04,0.4
OPTION_TYPE,P,C,P,C,C
issue_type,Stock option,Stock option,Others,Others,Others
TRADE_SIZE_binned,"(3,5]","(0,1]","(0,1]","(1,3]","(3,5]"
year_binned,2015,2015,2015,2015,2015
ttm_binned,<= 1,<= 1,<= 1,<= 1,<= 1
myn_binned,(0.9-1.1],(0.9-1.1],(0.7-0.9],(0.7-0.9],(0.9-1.1]
prox_q_binned,at quotes,outside,at quotes,at quotes,outside
mid,3.675,6.575,0.1,0.07,0.43


## Accuracy Calculation

In [18]:
# FIXME: Find better approach
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

result_dfs = []

for criterion in tqdm(criterions):
    results = []
    for classifier in tqdm(classifiers):
        res = (
            X_print.groupby([criterion])[["buy_sell", classifier]]
            .apply(
                lambda x: accuracy_score(x["buy_sell"].astype("int8"), x[classifier])
            )
            .mul(100)
            .rename(classifier)
        )
        #         acc_tot = accuracy_score(
        #             X_print["buy_sell"].astype("int8"), X_print[classifier]
        #         )

        #         res.loc["all"] = acc_tot * 100

        res.index.name = LUT_INDEX.get(criterion)
        results.append(res)

    # save aggregated results
    result_df = pd.concat(results, axis=1).T
    result_df.style.pipe(
        set_tex_style,
        caption=(f"long-tbd", "short-tbd"),
        label=f"{KEY.lower()}-{criterion.lower()}",
    )

    # store all result sets for later use
    result_dfs.append(result_df)


100%|██████████| 8/8 [02:40<00:00, 20.09s/it]


In [19]:
master = pd.concat(result_dfs, axis=1, keys=list(LUT_INDEX.values())).T


In [20]:
master


gbm                            \
                               gbm(classical) gbm(classical-retraining)   
Option Type       C                 62.889592                 65.781794   
                  P                 64.544991                 67.112672   
Security Type     Index option      56.308184                 56.614402   
                  Others            68.405530                 70.200752   
                  Stock option      61.866828                 65.027478   
Trade Size        (0,1]             61.886324                 64.923233   
                  (1,3]             62.455088                 65.420139   
                  (3,5]             62.683294                 65.649639   
                  (5,11]            65.941513                 68.272577   
                  >11               67.305363                 69.386769   
Year              2015              60.396806                 63.511360   
                  2016              63.737864                 66.222497   
                  2017              64.587031                 67.796378   
Time to Maturity  <= 1              64.456091                 66.953853   
                  (1-2]             64.596545                 67.148388   
                  (2-3]             62.979868                 65.494804   
                  (3-6]             61.803170                 65.104390   
                  (6-12]            61.849926                 65.082867   
                  > 12              55.148148                 60.880427   
Moneyness         <= 0.7 c          65.551797                 68.889840   
                  (0.7-0.9]         67.636359                 70.102678   
                  (0.9-1.1]         64.063302                 66.473410   
                  (1.1-1.3]         54.203956                 58.776086   
                  > 1.3             52.572955                 58.768938   
Location to Quote at mid            62.620494                 63.537872   
                  inside            68.627451                 70.588235   
                  at quotes         67.805369                 73.650919   
                  outside           62.609026                 64.733797   
                  unknown           71.533575                 72.890200   
All trades        all               63.662370                 66.403078   

                                                               \
                               gbm(classical-size-retraining)   
Option Type       C                                 73.577088   
                  P                                 74.375973   
Security Type     Index option                      58.684208   
                  Others                            77.549609   
                  Stock option                      72.731024   
Trade Size        (0,1]                             73.782177   
                  (1,3]                             74.408553   
                  (3,5]                             74.374525   
                  (5,11]                            73.690410   
                  >11                               73.596321   
Year              2015                              71.335358   
                  2016                              74.119968   
                  2017                              74.436152   
Time to Maturity  <= 1                              74.381920   
                  (1-2]                             74.301318   
                  (2-3]                             73.117570   
                  (3-6]                             72.901755   
                  (6-12]                            72.439952   
                  > 12                              71.093367   
Moneyness         <= 0.7 c                          74.141656   
                  (0.7-0.9]                         75.791937   
                  (0.9-1.1]                         74.309486   
                  (1.1-1.3]                         69.216210   
                

In [21]:
master.style.pipe(
    set_tex_style,
    caption=("master-long", "master-short"),
    label=f"{KEY}-master",
    bold_axis=0,
)


## Effective Spread 🚧

In [22]:
classifiers


[('gbm', 'gbm(classical)'),
 ('gbm', 'gbm(classical-retraining)'),
 ('gbm', 'gbm(classical-size-retraining)'),
 ('gbm', 'gbm(classical-size)'),
 ('gbm', 'gbm(ml)'),
 ('gbm', 'gbm(ml-retraining)'),
 ('gbm', 'gbm(semi-classical)'),
 ('classical', 'tick(ex)'),
 ('classical', 'quote(ex)'),
 ('classical', 'lr(ex)'),
 ('classical', 'emo(ex)'),
 ('classical', 'clnv(ex)'),
 ('classical',
  'trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)')]

In [45]:
X_print.head().T

index,39342191,39342190,39342189,39342188,39342187
buy_sell,-1,-1,1,-1,-1
TRADE_PRICE,3.5,6.38,0.13,0.04,0.4
OPTION_TYPE,P,C,P,C,C
issue_type,Stock option,Stock option,Others,Others,Others
TRADE_SIZE_binned,"(3,5]","(0,1]","(0,1]","(1,3]","(3,5]"
year_binned,2015,2015,2015,2015,2015
ttm_binned,<= 1,<= 1,<= 1,<= 1,<= 1
myn_binned,(0.9-1.1],(0.9-1.1],(0.7-0.9],(0.7-0.9],(0.9-1.1]
prox_q_binned,at quotes,outside,at quotes,at quotes,outside
mid,3.675,6.575,0.1,0.07,0.43


In [50]:
classifier = ("classical", "tick(ex)")
# rel_pred = (
#         effective_spread(X_print[classifier], X_print["TRADE_PRICE"], X_print["mid"], mode="relative")
#         * 100
#     )

nom = effective_spread(X_print[classifier], X_print["TRADE_PRICE"], X_print["mid"], mode="nominal")
rel = effective_spread(X_print[classifier], X_print["TRADE_PRICE"], X_print["mid"], mode="relative")

# calculate pred rel effective spread but not aggregated convert to %
es_pred = effective_spread(X_print[classifier], X_print["TRADE_PRICE"], X_print["mid"], mode="none")
eps_pred = np.empty(es_pred.shape)
np.divide(es_pred, X_print["mid"], out=eps_pred, where=X_print["mid"] != 0)

# calculate true rel effective spread but not aggregated, convert to %
es_true = effective_spread(X_print["buy_sell"], X_print["TRADE_PRICE"], X_print["mid"], mode="none")
eps_true = np.empty(es_true.shape)
np.divide(es_true, X_print["mid"], out=eps_true, where=X_print["mid"] != 0)

# quoted rel spread
# (ask -bid) / ask

wilcoxon_res  = wilcoxon(eps_pred, eps_true, nan_policy="omit", zero_method="zsplit")

res = pd.Series(
        {
            "nominal": nom * 100,
            "rel": rel * 100,
            'statistic':wilcoxon_res.statistic,
            'pvalue':wilcoxon_res.pvalue
        }
    )

print(res)

nominal      1.557282e+00
rel          1.080205e+00
statistic    2.228051e+13
pvalue       0.000000e+00
dtype: float64


In [49]:
res.to_frame()

,0
nominal,1.557282e-02
rel,1.080205e-02
statistic,2.228051e+13
pvalue,0.000000e+00


In [65]:
results = []

# calculate true rel effective spread but not aggregated, convert to %
es_true = effective_spread(X_print["buy_sell"], X_print["TRADE_PRICE"], X_print["mid"], mode="none")
eps_true = np.empty(es_true.shape)
np.divide(es_true, X_print["mid"], out=eps_true, where=X_print["mid"] != 0)

rel_true = effective_spread(X_print["buy_sell"], X_print["TRADE_PRICE"], X_print["mid"], mode="relative")
nom_true = effective_spread(X_print["buy_sell"], X_print["TRADE_PRICE"], X_print["mid"], mode="nominal")

for classifier in tqdm(classifiers):
    nom_pred = effective_spread(X_print[classifier], X_print["TRADE_PRICE"], X_print["mid"], mode="nominal")
    rel_pred = effective_spread(X_print[classifier], X_print["TRADE_PRICE"], X_print["mid"], mode="relative")

    # calculate pred rel effective spread but not aggregated convert to %
    es_pred = effective_spread(X_print[classifier], X_print["TRADE_PRICE"], X_print["mid"], mode="none")
    eps_pred = np.empty(es_pred.shape)
    np.divide(es_pred, X_print["mid"], out=eps_pred, where=X_print["mid"] != 0)

    # optional: quoted rel spread
    # (ask -bid) / ask

    wilcoxon_res  = wilcoxon(eps_pred, eps_true, nan_policy="omit", zero_method="zsplit")

    res = pd.Series(
            {
                "nom_pred": nom_pred * 100,
                "rel_pred": rel_pred * 100,
                "nom_true": nom_true * 100,
                "rel_true": rel_true * 100,
                'statistic':wilcoxon_res.statistic,
                'pvalue':wilcoxon_res.pvalue
            }, name=classifier
        )
    results.append(res)

results = pd.concat(results, axis=1)

100%|██████████| 13/13 [00:40<00:00,  3.12s/it]


In [67]:
results.T.style.format("{:.3f}")

## Diffs 🔄️

In [28]:
# classical baselines

base = master[
    [
        ("classical", "quote(ex)"),
        (
            "classical",
            "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)",
        ),
        (
            "classical",
            "trade_size(ex)->quote(best)->depth(best)->quote(ex)->depth(ex)->rev_tick(all)",
        ),
    ]
]

# my ml models
revised = master[
    [("gbm", "gbm(classical)"), ("gbm", "gbm(classical-size)"), ("gbm", "gbm(ml)")]
]


In [29]:
def combine_results(revised: pd.DataFrame, base: pd.DataFrame) -> pd.DataFrame:
    """
    Generate print layout like in Grauer et al.

    https://tex.stackexchange.com/questions/430283/table-with-numbers-in-parentheses-in-siunitx/430290#430290

    # see p. https://texdoc.org/serve/siunitx/0
    """
    # first, second layer of colum index
    c_1 = revised.columns.get_level_values(1)
    c_2 = ["nom"]
    midx = pd.MultiIndex.from_product([c_1, c_2])

    # copy data from revised add as (column, "nom")
    combo = pd.DataFrame(revised.values, index=revised.index, columns=midx)

    for i, mul_col in enumerate(combo.columns):

        combo[(mul_col[0], "pm")] = (combo[mul_col] - base.iloc[:, i]).round(2)
        combo.sort_index(axis=1, inplace=True)

    return combo


In [30]:
diff = combine_results(revised, base)

diff.style.to_latex(
    f"diff-{KEY}.tex",
    siunitx=True,
    position_float="centering",
    hrules=True,
    clines="skip-last;data",
    label=f"tab:diff-{KEY}",
    caption=(f"long-diff-{KEY}", f"short-diff-{KEY}"),
    convert_css=True,
)


In [31]:
diff


gbm(classical)        gbm(classical-size)  \
                                          nom     pm                 nom   
Option Type       C                 62.889592   9.59           71.895259   
                  P                 64.544991   9.77           72.892594   
Security Type     Index option      56.308184   4.21           57.640799   
                  Others            68.405530   9.47           76.385942   
                  Stock option      61.866828   9.84           70.962294   
Trade Size        (0,1]             61.886324   9.57           72.589837   
                  (1,3]             62.455088  10.22           73.006333   
                  (3,5]             62.683294  10.06           72.766890   
                  (5,11]            65.941513   8.94           71.771961   
                  >11               67.305363   9.62           71.307368   
Year              2015              60.396806   8.76           69.028053   
                  2016              63.737864   9.24           72.560975   
                  2017              64.587031  11.00           73.018742   
Time to Maturity  <= 1              64.456091  10.30           72.770627   
                  (1-2]             64.596545   9.97           72.885430   
                  (2-3]             62.979868   8.88           71.797757   
                  (3-6]             61.803170   8.49           71.302394   
                  (6-12]            61.849926   7.85           71.222635   
                  > 12              55.148148   4.43           68.602077   
Moneyness         <= 0.7 c          65.551797   7.61           71.843131   
                  (0.7-0.9]         67.636359  10.18           74.290189   
                  (0.9-1.1]         64.063302  10.12           72.990099   
                  (1.1-1.3]         54.203956   6.96           66.292506   
                  > 1.3             52.572955   6.09           63.007743   
Location to Quote at mid            62.620494  41.76           72.155374   
                  inside            68.627451  68.63           27.450980   
                  at quotes         67.805369   7.84           86.648398   
                  outside           62.609026   5.44           68.329476   
                  unknown           71.533575  52.17           71.828494   
All trades        all               63.662370   9.67           72.360838   

                                         gbm(ml)         
                                   pm        nom     pm  
Option Type       C              5.05  73.624730   6.77  
                  P              5.63  74.593715   7.33  
Security Type     Index option  -0.87  58.100126  -0.41  
                  Others         6.51  77.551217   7.67  
                  Stock option   4.93  72.919301   6.89  
Trade Size        (0,1]          4.01  74.242395   5.66  
                  (1,3]          4.35  74.813673   6.16  
                  (3,5]          3.94  74.557255   5.74  
                  (5,11]         8.57  73.383853  10.19  
                  >11            6.88  73.084679   8.66  
Year              2015           5.74  71.423694   8.13  
                  2016           5.60  74.235176   7.27  
                  2017           4.53  74.603733   6.12  
Time to Maturity  <= 1           5.81  74.560803   7.60  
                  (1-2]          4.97  73.865438   5.95  
                  (2-3]          4.12  72.946987   5.27  
                  (3-6]          4.04  72.719581   5.45  
                  (6-12]         3.95  73.093080   5.82  
                  > 12           4.01  72.039005   7.44  
Moneyness         <= 0.7 c       7.85  72.438812   8.45  
                  (0.7-0.9]      6.65  75.357081   7.71  
                  (0.9-1.1]      5.00  74.414466   6.43  
                  (1.1-1.3]      4.67  70.268926   8.65  
                  > 1.3          3.07  70.550826  10.61  
Location to Quote at mid         6.26  74.659713   8.76  
                  inside       -